In [1]:
from scipy.spatial import distance as dist 
from imutils import perspective 
from imutils import contours 
import numpy as np 
import imutils 
import cv2 
import tensorflow as tf

def midpoint(ptA, ptB):
	return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

def prepare(filepath):
    IMG_SIZE = 150
    img_array = cv2.imread(filepath,cv2.IMREAD_COLOR)
    img_array = img_array / 255
    resized_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return resized_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)


def get_stats(rads):
	vol = (4/3) * (22/7) * (rads**3)
	mass = vol*0.75
	print("Calories(Kcals) = ",mass*0.51)
	print("Water(g) = ",mass*0.862)
	print("Protein(g) = ",mass*0.006)
	print("Total Fat(g) = ",mass*0.005)
	print("Carbohydrate(g) = ",mass*0.116)
	print("Sugars(g) = ",mass*0.16)
	print("Protein(g) = ",mass*0.006)


model = tf.keras.models.load_model("Fruits_models.h5")

categories = ["Apple","Banana","Grape","Mango","Strawberry"]
predictions = []

File_Path = "app22.jpg"


img_arr = prepare(File_Path)
pred = model.predict(img_arr)
print(pred)
Fruit_type = (categories[np.argmax(pred)])

image = cv2.imread(File_Path)
image_org = cv2.resize(image,(500,500))
image = cv2.cvtColor(image_org, cv2.COLOR_BGR2RGB)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (7, 7), 0)


edged = cv2.Canny(gray, 50, 100)
edged = cv2.dilate(edged, None, iterations=1)
edged = cv2.erode(edged, None, iterations=1)


cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
	cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)

#(cnts, _) = contours.sort_contours(cnts)
pixelsPerMetric = None


for c in cnts:
	if cv2.contourArea(c) < 500:
		continue

	orig = image_org.copy()
	box = cv2.minAreaRect(c)
	box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
	box = np.array(box, dtype="int")

	
	box = perspective.order_points(box)
	cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 2)

	for (x, y) in box:
		cv2.circle(orig, (int(x), int(y)), 5, (0, 0, 255), -1)


	(tl, tr, br, bl) = box
	(tltrX, tltrY) = midpoint(tl, tr)
	(blbrX, blbrY) = midpoint(bl, br)


	(tlblX, tlblY) = midpoint(tl, bl)
	(trbrX, trbrY) = midpoint(tr, br)


	cv2.circle(orig, (int(tltrX), int(tltrY)), 5, (255, 0, 0), -1)
	cv2.circle(orig, (int(blbrX), int(blbrY)), 5, (255, 0, 0), -1)
	cv2.circle(orig, (int(tlblX), int(tlblY)), 5, (255, 0, 0), -1)
	cv2.circle(orig, (int(trbrX), int(trbrY)), 5, (255, 0, 0), -1)

	cv2.line(orig, (int(tltrX), int(tltrY)), (int(blbrX), int(blbrY)),
		(255, 0, 255), 2)
	cv2.line(orig, (int(tlblX), int(tlblY)), (int(trbrX), int(trbrY)),
		(255, 0, 255), 2)



	dA = dist.euclidean((tltrX, tltrY), (blbrX, blbrY))
	dB = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))

	
	pixel_to_cms = 0.0264583333
	dimA = dA * pixel_to_cms
	dimB = dB * pixel_to_cms


	cv2.putText(orig, "{:.1f}Cm".format(dimA)+Fruit_type,
		(int(tltrX - 15), int(tltrY - 10)), cv2.FONT_HERSHEY_SIMPLEX,
		0.65, (255, 255, 255), 2)
	cv2.putText(orig, "{:.1f}Cm".format(dimB)+Fruit_type,
		(int(trbrX + 10), int(trbrY)), cv2.FONT_HERSHEY_SIMPLEX,
		0.65, (255, 255, 255), 2)

	get_stats(min(dimA,dimB))

	cv2.imshow("Image", orig)
	cv2.waitKey(0)

c:\Users\tumur\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\tumur\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
[[0.78941566 0.00452397 0.00167845 0.19666502 0.00771681]]
Calories(Kcals) =  0.6358798038096838
Water(g) =  1.074761550752838
Protein(g) =  0.007480938868349221
Total Fat(g) =  0.006234115723624351
Carbohydrate(g) =  0.14463148478808494
Sugars(g) =  0.19949170315597922
Protein(g) =  0.007480938868349221


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
